# 

In [6]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be sav

/kaggle/input/baseline3chfinr/CHF_INR Historical Data.csv


In [7]:
import seaborn as sns
import numpy as np
import time
from sklearn.metrics import r2_score
import math
from sklearn import svm,preprocessing 
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error 
from sklearn.metrics import mean_absolute_error 
from numpy import array
import numpy as np
import numpy
%timeit
from sklearn.datasets import load_linnerud
from sklearn.multioutput import MultiOutputRegressor
from sklearn.linear_model import Ridge
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error 

import xgboost as xg
from sklearn.metrics import mean_squared_error 
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error

In [8]:
import random
from tensorflow.compat.v1.random import set_random_seed

random.seed(1)
numpy.random.seed(1)
set_random_seed(2)

In [9]:
df = pd.read_csv ("/kaggle/input/baseline3chfinr/CHF_INR Historical Data.csv") 
df.head()

,Date,Price,Open,High,Low,Vol.,Change %
0,01/01/2021,82.575,81.462,82.602,81.462,0.49K,0.12%
1,12/31/2020,82.480,83.006,83.119,82.459,30.42K,-0.59%
2,12/30/2020,82.967,83.033,83.237,82.673,40.00K,0.01%
3,12/29/2020,82.955,82.703,83.090,82.617,35.00K,0.32%
4,12/28/2020,82.692,82.705,82.832,82.410,34.13K,0.02%


In [10]:
df=df.loc[::-1]

In [11]:
df = df[['Open','High','Low','Price']]
df.head()

,Open,High,Low,Price
1566,63.379,63.816,63.359,63.740
1565,63.732,63.799,63.163,63.203
1564,63.131,63.221,62.693,62.901
1563,62.881,63.302,62.819,62.939
1562,62.922,62.941,62.176,62.385


In [12]:
df.rename(columns = {'Open':'open','High':'high','Low':'low','Price':'close'}, inplace = True)
df.head()

,open,high,low,close
1566,63.379,63.816,63.359,63.740
1565,63.732,63.799,63.163,63.203
1564,63.131,63.221,62.693,62.901
1563,62.881,63.302,62.819,62.939
1562,62.922,62.941,62.176,62.385


In [13]:
df["close-shifted"] = df.close.shift(1)
df["high-shifted"] = df.high.shift(1)
df["open-shifted"] = df.open.shift(1)
df["low-shifted"] = df.low.shift(1)

df.head()

,open,high,low,close,close-shifted,high-shifted,open-shifted,low-shifted
1566,63.379,63.816,63.359,63.740,NaN,NaN,NaN,NaN
1565,63.732,63.799,63.163,63.203,63.740,63.816,63.379,63.359
1564,63.131,63.221,62.693,62.901,63.203,63.799,63.732,63.163
1563,62.881,63.302,62.819,62.939,62.901,63.221,63.131,62.693
1562,62.922,62.941,62.176,62.385,62.939,63.302,62.881,62.819


In [14]:
df=df.dropna()
df.isna().sum()

open             0
high             0
low              0
close            0
close-shifted    0
high-shifted     0
open-shifted     0
low-shifted      0
dtype: int64

In [15]:
df=df[['open-shifted','high-shifted','low-shifted','close-shifted','close']]
df_without_scale=df

In [16]:

df.head(7)

,open-shifted,high-shifted,low-shifted,close-shifted,close
1565,63.379,63.816,63.359,63.740,63.203
1564,63.732,63.799,63.163,63.203,62.901
1563,63.131,63.221,62.693,62.901,62.939
1562,62.881,63.302,62.819,62.939,62.385
1561,62.922,62.941,62.176,62.385,61.537
1560,62.366,62.426,61.326,61.537,61.482
1559,61.532,61.644,61.039,61.482,61.167


In [17]:
df=df.dropna()

In [18]:
df.isna().sum()

open-shifted     0
high-shifted     0
low-shifted      0
close-shifted    0
close            0
dtype: int64

In [19]:
df.columns

Index(['open-shifted', 'high-shifted', 'low-shifted', 'close-shifted',
       'close'],
      dtype='object')

In [20]:

# شيفت البرايس ب 1 لتكوين عامود التارجت

In [21]:
df=df.dropna()
df.isna().sum()

open-shifted     0
high-shifted     0
low-shifted      0
close-shifted    0
close            0
dtype: int64

In [22]:
close_3thdDayClose=df[["close-shifted",'close']]
close_3thdDayClose.head(10)

,close-shifted,close
1565,63.740,63.203
1564,63.203,62.901
1563,62.901,62.939
1562,62.939,62.385
1561,62.385,61.537
1560,61.537,61.482
1559,61.482,61.167
1558,61.167,60.756
1557,60.756,60.958
1556,60.958,72.316


In [23]:
data=df.to_numpy()
df.head(3)

,open-shifted,high-shifted,low-shifted,close-shifted,close
1565,63.379,63.816,63.359,63.740,63.203
1564,63.732,63.799,63.163,63.203,62.901
1563,63.131,63.221,62.693,62.901,62.939


In [24]:
df.shape

(1566, 5)

In [25]:
x_close=df

In [26]:
percentage = 90
split=x_close.shape[0]*percentage//100
split

1409

In [27]:
df_train=x_close.iloc[:split,:]
df_train.shape

(1409, 5)

In [28]:
df_test=x_close.iloc[split:,:]
df_test.shape

(157, 5)

In [29]:
def split_sequences(sequences, n_steps_in, n_steps_out):
    X, y = list(), list()
    for i in range(len(sequences)):
        # find the end of this pattern
        end_ix = i + n_steps_in
        out_end_ix = end_ix + n_steps_out-1
        # check if we are beyond the dataset
        if out_end_ix > len(sequences):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1:out_end_ix, -1]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)

In [30]:
dataset_train=df_train.values
dataset_test=df_test.values


In [31]:
n_steps_in, n_steps_out = 7,7
# convert into input/output
train_x, train_y = split_sequences(dataset_train, n_steps_in, n_steps_out)
# the dataset knows the number of features, e.g. 2


In [32]:
df_train.head(17)

,open-shifted,high-shifted,low-shifted,close-shifted,close
1565,63.379,63.816,63.359,63.740,63.203
1564,63.732,63.799,63.163,63.203,62.901
1563,63.131,63.221,62.693,62.901,62.939
1562,62.881,63.302,62.819,62.939,62.385
1561,62.922,62.941,62.176,62.385,61.537
1560,62.366,62.426,61.326,61.537,61.482
1559,61.532,61.644,61.039,61.482,61.167
1558,61.466,61.631,60.925,61.167,60.756
1557,61.154,61.332,60.660,60.756,60.958
1556,60.731,61.242,60.689,60.958,72.316


In [36]:
train_x.shape

(1397, 7, 4)

In [37]:
train_x[:1]

array([[[63.379, 63.816, 63.359, 63.74 ],
        [63.732, 63.799, 63.163, 63.203],
        [63.131, 63.221, 62.693, 62.901],
        [62.881, 63.302, 62.819, 62.939],
        [62.922, 62.941, 62.176, 62.385],
        [62.366, 62.426, 61.326, 61.537],
        [61.532, 61.644, 61.039, 61.482]]])

In [38]:
train_y

array([[61.167, 60.756, 60.958, ..., 72.152, 70.083, 70.398],
       [60.756, 60.958, 72.316, ..., 70.083, 70.398, 71.573],
       [60.958, 72.316, 72.152, ..., 70.398, 71.573, 70.705],
       ...,
       [78.017, 77.926, 77.851, ..., 77.892, 78.184, 78.038],
       [77.926, 77.851, 78.266, ..., 78.184, 78.038, 78.103],
       [77.851, 78.266, 77.892, ..., 78.038, 78.103, 78.352]])

In [39]:
# convert into input/output
test_x, test_y = split_sequences(dataset_test, n_steps_in, n_steps_out)
# the dataset knows the number of features, e.g. 2


In [40]:
train_x=train_x.reshape(train_x.shape[0],train_x.shape[1]*train_x.shape[2])
train_x.shape

(1397, 28)

In [41]:
train_y=train_y.reshape(train_y.shape[0],train_y.shape[1])
train_y.shape

(1397, 7)

In [42]:
test_x=test_x.reshape(test_x.shape[0],test_x.shape[1]*test_x.shape[2])
test_x.shape

(145, 28)

In [43]:
test_y=test_y.reshape(test_y.shape[0],test_y.shape[1])
test_y.shape

(145, 7)

# SVR

##  Results

In [44]:

regr = MultiOutputRegressor(SVR(kernel='linear',C=1e1,epsilon=0.0001,verbose=False,cache_size=15000)).fit(train_x, train_y)
y_predict4=regr.predict(test_x)
#%%timeit
print('------------------------------------------------------------------')
result4=mean_absolute_error(test_y, y_predict4)
result4=round(result4, 6)
result4='{:f}'.format(result4)
print('MAE linear',result4)
result5=mean_squared_error(test_y, y_predict4)
result5=round(result5, 6)
result5='{:f}'.format(result5)
print('MSE linear',result5)
RE=round(r2_score(test_y, y_predict4),6)
result1='{:f}'.format(RE)
print('r2',result1)

------------------------------------------------------------------
MAE linear 0.627944
MSE linear 0.642800
r2 0.505291


In [45]:

regr = MultiOutputRegressor(SVR(kernel='rbf',C=1e1,epsilon=0.0001,verbose=False,cache_size=15000)).fit(train_x, train_y)
y_predict4=regr.predict(test_x)
#%%timeit
print('------------------------------------------------------------------')
result4=mean_absolute_error(test_y, y_predict4)
result4=round(result4, 6)
result4='{:f}'.format(result4)
print('MAE rbf',result4)
result5=mean_squared_error(test_y, y_predict4)
result5=round(result5, 6)
result5='{:f}'.format(result5)
print('MSE rbf',result5)
RE=round(r2_score(test_y, y_predict4),6)
result1='{:f}'.format(RE)
print('r2',result1)

------------------------------------------------------------------
MAE rbf 5.480497
MSE rbf 38.579326
r2 -28.752128


In [46]:

regr = MultiOutputRegressor(SVR(kernel='poly',C=1e1,epsilon=0.001,degree=3,verbose=False,cache_size=15000)).fit(train_x, train_y)
y_predict4=regr.predict(test_x)
#%%timeit
print('----------------------------poly 3--------------------------------------')
result4=mean_absolute_error(test_y, y_predict4)
result4=round(result4, 6)
result4='{:f}'.format(result4)
print('MAE Poly 3',result4)
result5=mean_squared_error(test_y, y_predict4)
result5=round(result5, 6)
result5='{:f}'.format(result5)
print('MSE Poly 3',result5)
RE=round(r2_score(test_y, y_predict4),6)
result1='{:f}'.format(RE)
print('r2',result1)

----------------------------poly 3--------------------------------------
MAE Poly 3 453.687349
MSE Poly 3 309146.808268
r2 -239029.293194


In [47]:

regr = MultiOutputRegressor(SVR(kernel='poly',C=1e1,epsilon=0.0001,degree=2,verbose=False,cache_size=15000)).fit(train_x, train_y)
y_predict4=regr.predict(test_x)
#%%timeit
print('----------------------------poly 2--------------------------------------')
result4=mean_absolute_error(test_y, y_predict4)
result4=round(result4, 6)
result4='{:f}'.format(result4)
print('MAE Poly 2',result4)
result5=mean_squared_error(test_y, y_predict4)
result5=round(result5, 6)
result5='{:f}'.format(result5)
print('MSE Poly 2',result5)
RE=round(r2_score(test_y, y_predict4),6)
result1='{:f}'.format(RE)
print('r2',result1)

----------------------------poly 2--------------------------------------
MAE Poly 2 0.940578
MSE Poly 2 1.445706
r2 -0.114924


# XGBoost

In [48]:

train_y=train_y.reshape((train_y.shape[0],n_steps_out))
train_y.shape

(1397, 7)

In [49]:

# Instantiation
xgb_r = xg.XGBRegressor( n_estimators =60,max_depth=80)
  
# Fitting the model
xgb_r.fit(train_x, train_y)
  
# Predict the model
pred = xgb_r.predict(test_x)
print('MAE xGBOOST',round(mean_absolute_error(test_y, pred),6))
print('MSE xGBOOST',round(mean_squared_error(test_y, pred),6))

print('R2 xGBOOST',round(r2_score( test_y,pred),6))

MAE xGBOOST 3.270594
MSE xGBOOST 12.300774
R2 xGBOOST -8.498904
